In [8]:
from fsl.wrappers import bet
from pathlib import Path
import subprocess as sp
import copy
import shutil
from tqdm import tqdm
import importlib
import sub
importlib.reload(sub) 



<module 'sub' from '/Volumes/T7/MAP/sub.py'>

# **Create images and derivates folders**

In [2]:
CWD  = Path.cwd()
data_dir = CWD / "raw_data"
data = CWD / "images"
procc = CWD /"derivates"
data.mkdir(exist_ok=True)
procc.mkdir(exist_ok=True)
subjects = []
for file in data_dir.glob("*.nii.gz"):
    name = file.name
    subject = Path("images",f"sub_{name.split('_')[1]}")
    procc_subject = procc / f"sub_{name.split('_')[1]}"
    subject.mkdir(exist_ok=True)
    procc_subject.mkdir(exist_ok=True)
    anat = subject / "anat"
    func = subject / "func"
    anati = procc_subject / "anat"
    funci = procc_subject / "func"
    c = [anat, func, anati, funci]
    l = [anat, func, anati, funci]
    for i in c : 
        hypo = i / "hyper"
        hyper = i / "hypo"
        l.append(hypo)
        l.append(hyper)
    for d in l:
        d.mkdir(exist_ok=True)
    if subject not in subjects:
        subjects.append(subject)
    

In [3]:

for mri in tqdm(data_dir.glob("*.nii.gz")):
    name = mri.name
    dec = name.split("_")
    sub = dec[1]
    cond = dec[2]
    type = dec[3].split(".")[0]
    if type == "A":
        type = "T1w"
        anat = True
    else :
        anat = False
        type = f"run{type}"
    if cond == "HYPO":
        hypo = True
    else:
        hypo = False
    new_file = Path(f"sub_{sub}_{cond}_{type}.nii.gz")
    
    if anat:
        if hypo :
            dst = data / f"sub_{sub}" / "anat" / "hypo" / new_file
        else:
            dst = data / f"sub_{sub}" / "anat" / "hyper" / new_file
    else:
        if hypo:
            dst = data / f"sub_{sub}" / "func" / "hypo" / new_file
        else:
            dst = data / f"sub_{sub}" / "func" / "hyper" / new_file
    shutil.copy(str(mri), str(dst))
print("DONE")


175it [00:04, 35.90it/s]

DONE


# **Create analysis Folder**


In [9]:
CWD = Path.cwd()
CAP_dir_nogsr = CWD / "cap_analysis_nogsr"
CAP_dir_nogsr.mkdir(exist_ok=True)
CAP_dir_withgsr = CWD / "cap_analysis_withgsr"
CAP_dir_withgsr.mkdir(exist_ok=True)
cap_dir_names = [CAP_dir_nogsr, CAP_dir_withgsr]


runs = [f"R{i}" for i in range(1, 5)]
conditions = ["hyper", "hypo"]
derivates = CWD / "derivates"
# print(type(derivates), derivates)
def create_dir(CAP_dir_name):
    for sub in derivates.iterdir():
        if "sub_" in sub.name:
            for cond in conditions : 
                for r in runs : 
                    dir = CAP_dir_name / f"{sub.name}_{cond}_{r}"
                    dir.mkdir(exist_ok=True)
                    anat = dir / "anat"
                    func = dir / "func"
                    anat.mkdir(exist_ok=True)
                    func.mkdir(exist_ok=True)

for i in cap_dir_names:
    create_dir(i)


**Move NIfTI files in 3D format**

In [6]:
full_sub = {}
cwd = Path.cwd()
sub_folder = cwd / "derivates"
for folder_name in sub_folder.iterdir():
    if folder_name.is_dir() and folder_name.name.startswith("sub_"):
        if folder_name.name == "sub_025":  # REMOVE SUB_25 he has no hypo data
            continue
        subject_name = folder_name.name
        full_sub[subject_name] = sub.Subject(subject_name)

NameError: name 'sub' is not defined

In [5]:


def split_fmri_4d_to_3d(input_file, output_dir):
    input_file = Path(input_file)
    output_dir = Path(output_dir)

    output_prefix = output_dir / Path(input_file.stem).stem  
    cmd = ["fslsplit", str(input_file), str(output_prefix) + "_", "-t"]

    print(f"Splitting {input_file.name} → {output_dir}")
    sp.run(cmd, check=True)
    print("✅ Done")


split_fmri_4d_to_3d("/Volumes/T7/MAP/derivates/sub_001/func/hyper/runR1/sub_001_HYPER_runR1_mc_bet_clean_withgsr_mni_smooth.nii.gz",
                    "/Volumes/T7/MAP/cap_analysis_withgsr/sub_001_hyper_R1/func")



Splitting sub_001_HYPER_runR1_mc_bet_clean_withgsr_mni_smooth.nii.gz → /Volumes/T7/MAP/cap_analysis_withgsr/sub_001_hyper_R1/func
✅ Done
